# First steps

ConFlowGen has been developed with Jupyter Notebooks in mind.
While it is also possible to write your code as pure Python Scripts (see e.g. the
[Python Script examples](https://github.com/1kastner/conflowgen/tree/main/examples/Python_Script)),
Jupyter Notebooks allow to mix the code with explanatory texts, LaTeX formulas, etc. in a single document.
In addition, the generated visualisations neatly blend in.

For newcomers it is suggested to download the
[Anaconda distribution](https://www.anaconda.com/)
and first to familiarize yourself with the newly installed tools.
Then, it is suggested to create a separate environment as described for the
[Jupyter Notebook examples](https://github.com/1kastner/conflowgen/tree/main/examples/Jupyter_Notebook).
During the environment set up, ConFlowGen is automatically installed.
Then, you can start JupyterLab
[from Anaconda Navigator](https://docs.anaconda.com/anaconda/navigator/index.html)
or
[from the CLI](https://jupyterlab.readthedocs.io/en/stable/getting_started/starting.html).
Please ensure that you are in the correct conda environment (it should be `conflowgen`) when you work.

## Prerequisites

To start, we first import ConFlowGen as a module.

In [ ]:
import os
import datetime

import pandas as pd

import conflowgen

Then, we set up the logger

In [ ]:
logger = conflowgen.setup_logger(
    logging_directory="./data/logger",  # use subdirectory relative to Jupyter Notebook
    format_string="%(message)s"  # only show log messages, discard timestamp etc.
)

## Database selection

Now, we select a database to work in.

In [ ]:
database_chooser = conflowgen.DatabaseChooser(
    sqlite_databases_directory="./data/db"  # use subdirectory relative to Jupyter Notebook
)
demo_file_name = "my_demo.sqlite"

database_chooser.create_new_sqlite_database(demo_file_name, overwrite=True)

## General settings

In [ ]:
container_flow_generation_manager = conflowgen.ContainerFlowGenerationManager()
container_flow_generation_manager.set_properties(
    name="Demo file",
    start_date=datetime.datetime.now().date(),
    end_date=datetime.datetime.now().date() + datetime.timedelta(days=21)
)

## Creating schedules

In [ ]:
port_call_manager = conflowgen.PortCallManager()

At first we define a name for our new feeder liner service.

In [ ]:
feeder_service_name = "LX050"

In [ ]:
port_call_manager.add_large_scheduled_vehicle(
    vehicle_type=conflowgen.ModeOfTransport.feeder,
    service_name=feeder_service_name,
    vehicle_arrives_at=datetime.date(2021, 7, 9),
    vehicle_arrives_at_time=datetime.time(11),
    average_vehicle_capacity=800,
    average_moved_capacity=100,
    next_destinations=[
        ("DEBRV", 0.4),  # 40% of the containers go here...
        ("RULED", 0.6)   # and the other 60% of the containers go here.
    ]
)

Following the same principle and structure we can also add schedules for trains and deep sea vessels:

In [ ]:
port_call_manager.add_large_scheduled_vehicle(
    vehicle_type=conflowgen.ModeOfTransport.train,
    service_name="JR03A",
    vehicle_arrives_at=datetime.date(2021, 7, 12),
    vehicle_arrives_at_time=datetime.time(17),
    average_vehicle_capacity=90,
    average_moved_capacity=90,
    next_destinations=None  # Here we don't have containers that need to be grouped by destination
)

In [ ]:
port_call_manager.add_large_scheduled_vehicle(
    vehicle_type=conflowgen.ModeOfTransport.deep_sea_vessel,
    service_name="LX050",
    vehicle_arrives_at=datetime.date(2021, 7, 10),
    vehicle_arrives_at_time=datetime.time(19),
    average_vehicle_capacity=16000,
    average_moved_capacity=150,  # for faster demo
    next_destinations=[
        ("ZADUR", 0.3),  # 30% of the containers go to ZADUR...
        ("CNSHG", 0.7)   # and the other 70% of the containers go to CNSHG.
    ]
)

## Generate the data

In [ ]:
container_flow_generation_manager.generate()

## Export the data

The data is exported with two lines of code.

In [ ]:
export_container_flow_manager = conflowgen.ExportContainerFlowManager()
path_to_exported_data = export_container_flow_manager.export("first_steps", "./data/export", overwrite=True)

## Examining the exported data

The CSV files we can open e.g. with pandas.
Here, the tabular data is presented.

In [ ]:
df_containers = pd.read_csv(
    os.path.join(path_to_exported_data, "containers.csv"),
    index_col="id",
    dtype={
        "delivered_by_truck": "Int64",
        "picked_up_by_truck": "Int64",
        "delivered_by_vehicle": "Int64",
        "picked_up_by_vehicle": "Int64",
        "destination_sequence_id": "Int64"
    }
)
df_containers

The column `delivered_by` contains the vehicle type a container is delivered by.
For trucks, the column `delivered_by_truck` mentions the corresponding ID of the vehicle that is stored in the separate table `trucks.csv`.
The same scheme applies to `picked_up_by` and `picked_up_by_truck`.

In [ ]:
df_trucks = pd.read_csv(
    os.path.join(path_to_exported_data, "trucks.csv"),
    index_col="id"
)
df_trucks

Here, we can see which trucks deliver a container, pick up a container, and for which time their arrivals are realized.

All vehicle types except trucks are considered vehicles that typically arrive according to a schedule and which move larger amounts of containers at once.
Thus, the table has a slightly different shape.
They all have ids from the same ID range and for each vehicle type the ids might be non-consecutive.
So e.g. for trains, in the column `delivered_by_vehicle` in the container table the ID is mentioned that we can look up in the table of `trains.csv`.

In [ ]:
df_trains = pd.read_csv(
    os.path.join(path_to_exported_data, "trains.csv"),
    index_col="id"
)
df_trains

Corresponding CSV files exist for the other vehicles as well.